# Regression model with Keras

### Part B. Building a baseline model with normalized data

Importing necessary libraries and loading data

In [1]:
import pandas as pd
import numpy as np

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

In [4]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [5]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [6]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

##### Now we normalize data

In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Installing tenzor flow and Keras

In [8]:
pip install keras-tuner --upgrade

Requirement already up-to-date: keras-tuner in c:\users\sdovl\anaconda3\lib\site-packages (1.0.3)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import keras_tuner as kt
from tensorflow import keras

Creating regression model

In [11]:
from keras.models import Sequential
from keras.layers import Dense

In [13]:
n_cols = predictors_norm.shape[1] 

In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [15]:
# build the model
model = regression_model()

###### 1. splitting data on train and test

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

###### 2.training model on train data

In [17]:
# fit the model
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
23/23 [==============================] - 17s 859us/step - loss: 1595.3495
Epoch 2/50
23/23 [==============================] - 0s 1ms/step - loss: 1611.4898
Epoch 3/50
23/23 [==============================] - 0s 2ms/step - loss: 1650.3280
Epoch 4/50
23/23 [==============================] - 0s 2ms/step - loss: 1598.8536
Epoch 5/50
23/23 [==============================] - 0s 1ms/step - loss: 1542.9180
Epoch 6/50
23/23 [==============================] - 0s 6ms/step - loss: 1538.1782
Epoch 7/50
23/23 [==============================] - 0s 2ms/step - loss: 1454.3324
Epoch 8/50
23/23 [==============================] - 0s 1ms/step - loss: 1546.2815
Epoch 9/50
23/23 [==============================] - 0s 1ms/step - loss: 1531.3397
Epoch 10/50
23/23 [==============================] - 0s 1ms/step - loss: 1440.9434
Epoch 11/50
23/23 [==============================] - 0s 1ms/step - loss: 1509.9453
Epoch 12/50
23/23 [==============================] - 0s 2ms/step - loss: 1458.7554
Epoch 13/5

###### 3. Evaluating model on test data

In [18]:
loss_val = model.evaluate(X_test, y_test)
loss_val

10/10 [==============================] - 0s 888us/step - loss: 358.4474


358.44744873046875

In [19]:
y_pred = model.predict(X_test)

###### computing mse between the predicted and actual values

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
mse = mean_squared_error(y_test, y_pred)
mse

358.44746983563147

###### 4. Repeating steps 1-3 50 times and create list of 50 mse

In [22]:
number_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, number_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    loss_val = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mse)
    print(" mse "+str(i+1)+": "+str(mse))


 mse 1: 149.3398837000025
 mse 2: 135.60617276143924
 mse 3: 93.42338414074482
 mse 4: 83.02193671487731
 mse 5: 71.66019574466384
 mse 6: 70.65432206459306
 mse 7: 60.9132872456453
 mse 8: 39.11641383074567
 mse 9: 43.247313010887865
 mse 10: 38.60486713517611
 mse 11: 39.70798960840193
 mse 12: 35.830687489195086
 mse 13: 43.16508951376625
 mse 14: 41.910084102999235
 mse 15: 32.32424896640628
 mse 16: 31.610789177718768
 mse 17: 35.127877330135604
 mse 18: 34.01770639042536
 mse 19: 33.85844039476287
 mse 20: 33.21762787666994
 mse 21: 33.13374274472334
 mse 22: 32.4212640545459
 mse 23: 31.859429630693413
 mse 24: 32.62420503056948
 mse 25: 34.55732166851751
 mse 26: 34.78864473483407
 mse 27: 30.706372856463418
 mse 28: 29.339855713012547
 mse 29: 37.74682305834182
 mse 30: 35.36247597308794
 mse 31: 30.631827798575248
 mse 32: 30.554634105042542
 mse 33: 28.869684396435744
 mse 34: 32.747831715290104
 mse 35: 33.38808574513308
 mse 36: 38.04879736301499
 mse 37: 29.44290359913849

###### 5.Finding mean and variance of MSE

In [23]:
mse = np.array(mse)
mean = np.mean(mse)
standard_deviation = np.std(mse)

print('\n')
print("The mean and standard deviation of " +str(number_mean_squared_errors) + " mean squared errors of normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))



The mean and standard deviation of 50 mean squared errors of normalized data. Total number of epochs for each training is: 50

Mean: 32.73596358080097
Standard Deviation: 0.0


## We can notice that in case of normalized data mean of MES significally reduced that indicate that model in part B is better describe our datas